Importing modules

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score
from Layer import *
from network import Network
from activation_func import tanh, tanh_prime, sigmoid, sigmoid_prime, softmax, softmax_prime, relu, relu_prime
from loss_func import mse, mse_prime, cross_entropy, cross_entropy_prime
from data_func import vectorize_labels, k_fold, import_data
from performance_func import plot_error, plot_confusion_matrix
from network_queue import Queue


Importing data

In [3]:

# import data and initialize seed
np.random.seed(10)

# import data
training_size = 5000
normalize = True
training, labels, test, original_test_labels, test_labels = import_data(size=training_size, normalize=normalize)

# intializing queue
queue = Queue(training, labels, test, test_labels, original_test_labels)

#setting up the models

architectures = [[20],[15]]
params =  {"epochs": 5,
            "learning_rate": 5e-3,
            "batch_size": 1,
            "momentum": False,
            "weight_decay": 0.01}

for layers in architectures:
    queue.add(
        description="architecture "+str(layers)+" training_size "+str(training_size),
        netparams={
        "hidden_layers": layers,
        "features": 784,
        "output_classes": 10,
        "activation": relu,
        "activation_prime": relu_prime,
        "loss_activation": softmax,
        "loss_activation_prime": softmax_prime,
        "loss": cross_entropy,
        "loss_prime": cross_entropy_prime
    },
    folds=5,
    params=params)
#     queue.add(
#         description="architecture: "+str(layers)+" training_size: "+str(training_size),
#         netparams={
#         "hidden_layers": layers,
#         "features": 784,
#         "output_classes": 10,
#         "activation": sigmoid,
#         "activation_prime": sigmoid_prime,
#         "loss_activation": softmax,
#         "loss_activation_prime": softmax_prime,
#         "loss": cross_entropy,
#         "loss_prime": cross_entropy_prime
#     },
#     folds=5,
#     params=params)

#execute queue
results = queue.execute()

Fitting model 1/10
epoch 1/5   training error=3.679974  validation error=3.119066 validation accuracy=0.200000 ETA=0:00:00
epoch 2/5   training error=2.005017  validation error=2.506510 validation accuracy=0.300000 ETA=0:00:00
epoch 3/5   training error=1.471187  validation error=2.272865 validation accuracy=0.400000 ETA=0:00:00
epoch 4/5   training error=1.106831  validation error=2.013760 validation accuracy=0.450000 ETA=0:00:00
epoch 5/5   training error=0.887517  validation error=1.864514 validation accuracy=0.500000 ETA=0:00:00
Average epoch computational time:  0.011305952072143554
Fitting model 2/10
epoch 1/5   training error=3.709971  validation error=3.425584 validation accuracy=0.100000 ETA=0:00:00
epoch 2/5   training error=2.377229  validation error=2.954744 validation accuracy=0.150000 ETA=0:00:00
epoch 3/5   training error=1.847922  validation error=2.753283 validation accuracy=0.200000 ETA=0:00:00
epoch 4/5   training error=1.563008  validation error=2.645253 validation 